In [16]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline

# 如何制作数据集
* 如果要制造一台自动驾驶汽车，那么必须首先实现车辆探测系统，这样才能避免与其它车辆相撞。
* 为了采集数据，可以在汽车上面安装一个照相机，然后开着车在马路上溜达，照相机会每秒拍一张照片，其实百度地图里面的全景模式也是通过这种方式采集照片的
* 采集到照片后，需要给这些相片打上标签，如下图所示：
    <img src="nb_images/box_label.png" style="width:500px;height:250;">
<caption><center> <u> **图 1** </u>: 给图片打标签<br> </center></caption>

图中的p<sub>c</sub>表示有物体的概率,1就表示100%有物体，0表示绝对没有物体。c表示物体的种类，是行人还是汽车...其实有两种表示方法，假如你要识别80种物体，那么一种方法是用1到80的整数来表示不同的种类，上图中c=3就表示是汽车；另一种方法是用一个有80个元素的向量的表示，也就是说每个元素代表一个种类，如果值是1就表示肯定是这个种类，如果是0就表示绝对不可能是这个种类，在前面的文章中我们主要使用的是这种方法。而在本次实战编程中，我们会两种方法都用，具体取决于在某个步骤时用哪个方法比较方便。

## Yolo探测法
YOLO 是一个非常流行的算法，因为它不仅精度高，而且由于计算量较小还可以被用于实时探测

## YOLO模型细节
使用github已经训练好的YOLO模型：
    * 这个模型的输入维度是（m, 608, 608, 3）。
    m是图片的个数，608是图片的像素值
    * 它会输入识别出来的物体类别以及它的边框坐标，每个边框都对应着6个元素
    （pc, bx, by, bn, bw, c）。当然，如果c是用一个包含80个元素的向量来表示的话，那么每个边框就对应着85个元素
    * 因为这个YOLO模型使用来5个anchor boxes，并且图片被分成来19个格子，所以这个模型最终输出的维度是（m, 19, 19, 5, 85）

大致维度情况就是（m，608， 608，3）-> YOLO模型（CNN网络）-> (m, 19, 19,5, 85),如下图所示：
    <img src="nb_images/architecture.png" style="width:700px;height:400;">
<caption><center> <u> **图 2** </u>: <br> </center></caption>

因为这个模型使用了5个anchor boxes（大小不同的框框），所以每个小格子都对应了5组包含85个元素的向量，所以维度是(19, 19, 5, 85)。而为了简单起见，会把后面两个维度扁平化在一起，所以维度就变成了(19, 19, 425)。

<img src="nb_images/flatten.png" style="width:700px;height:400;">
<caption><center> <u> **图 3** </u>: **扁平化后面两个维度**<br> </center></caption>

将p<sub>c</sub>与后面的c相乘，就会得到每个种类的“最终”概率值。p<sub>c</sub>  x  c=有物体的概率 x 某个种类的概率。

<img src="nb_images/probability_extraction.png" style="width:700px;height:400;">
<caption><center> <u> **图 4** </u>: **图中计算结果表明box1有物体并且是汽车的概率为0.44.**<br> </center></caption>

我们可以用假设为格子填充不同颜色的方式来辅助我们理解YOLO算法的一些内部过程。不同颜色代表着不同种类的物体。
某些格子被填充为了橙色，就代表YOLO算法在这格子中统计得到的数据后发现，关联的5个anchor boxes中p<sub>c</sub>与后面的c相乘得到的结果是汽车的概率最高。

<img src="nb_images/proba_map.png" style="width:300px;height:300;">
<caption><center> <u> **图 5** </u><br> </center></caption>


另外一种辅助理解的方式是画出YOLO探测出的边框。如下图所示。每个格子都对应着5个anchor boxes，所以每个格子都有5个边框被探测出来,边框的颜色也代表着不同种类。当然，下图中只画出了一些有物体概率比较高的格子对应的边框。其实总边框应该有1805个，19x19x5 = 1805。

<img src="nb_images/anchor_map.png" style="width:200px;height:200;">
<caption><center> <u> **图 6** </u><br> </center></caption>

虽然上图中只画出了部分高概率的边框，但是还是太多了。实践中，我们会先去掉哪些概率小的边框，然后再使用非最大值抑制算法来去掉哪些重叠的边框。

## 过滤掉概率小的边框

In [24]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold =.6):
    """
    参数：
    box_confidence -- 装载着每个边框的pc，维度是（19，19，5，1）
    boxes -- 装载着每个边框的bx，by，bh，bw，维度是（19，19，5，4）
    box_class_probs -- 装载着每个边框的80个种类的概率，维度是（19，19，5，80）
    threshold -- 阈值，概率低于这个值的边框会被过滤掉
    
    返回值：
    scores -- 装载保留下的那些边框的概率，维度是（None，）
    boxes -- 装载保留下的那些边框的坐标（bx，by，bh，bw），维度是（None, 4）
    classes -- 装载保留下的那些边框的种类的索引，维度是（None，）
    
    返回值维度中的None，因为不知道有多少个边框被过滤掉了，有多少被保留下了
    """
    
    #将px和c相乘
    box_scores = np.multiply(box_confidence, box_class_probs)
    
    #之前将keras as K
    box_classes = K.argmax(box_scores,axis=-1)#获取改了吧最大的那个种类的索引
    box_class_scores =K.max(box_scores, axis=-1)#获取概率最大的那个种类的概率值
    
    #采集一个过滤器。当某个种类的概率值大雨等于阈值时
    #对应这个种类的filtering_mask中的位置就是True，否则为False
    #所以filtering_mask 就是[False, True, .., False, True]这种形式
    filtering_mask =K.greater_equal(box_class_scores, threshold)
    
    #用完上面的过滤器来过滤掉那些小概率的边框
    #过滤完成后，scores和boxes， classes里面就只装载来概率大的边框的概率值和坐标以及种类索引
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes,filtering_mask)
    classes =tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

In [25]:
with tf.Session() as test_a:
    box_confidence = tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
    box_class_probs = tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.5)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.shape))
    print("boxes.shape = " + str(boxes.shape))
    print("classes.shape = " + str(classes.shape))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
scores[2] = 10.750582
boxes[2] = [ 8.426533   3.2713668 -0.5313436 -4.9413733]
classes[2] = 7
scores.shape = (?,)
boxes.shape = (?, 4)
classes.shape = (?,)


## 非最大值抑制 Non-max suppression
上面使用阈值过滤掉了一些小概率边框，但是依然有很多重叠的边框，这一步我们将只用非最大抑制Non-max suppression（NMS）技术来过滤掉那些重叠的边框

<img src="nb_images/non-max-suppression.png" style="width:500px;height:400;">
<caption><center> <u> **图 7** </u>: 左图中3个边框预测的其实都是同一辆车，通过NMS后，去掉重叠的边框，只保留概率值最大的那个，得到右图 <br> </center></caption>

NMS的主要步骤如下：
    1.找到概率值最高的一个边框
    2.找到与这个边框高度重叠的其它边框，也就是交并比很高的其它边框，然后过滤掉它们，当然，交并比到底多高才过滤掉，为此会给定一个阈值iou_threshold.
    3.重复进行上面两步

In [26]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes=10, iou_threshold =.5):
    """
    参考：
    scores -- 前面yolo_filter_boxes 函数保留下的那些边框概率值，维度是（None,）
    boxes -- 前面yolo_filter_boxes 函数保留下的那些边框的坐标（bx, by, bh, bw), 维度是（None,4）
    classes --前面yolo_filter_boxes 函数保留下的那些边框的种类的索引，维度是（None，）
    max_boxes -- 最多想要保留多少个边框
    iou_threshold -- 交并比，这是一个阈值，也就是说交并比大雨这个阈值的边框才会被进行非最大值抑制处理
    
    returns:
    scores -- NMS保留下的那些边框的概率值，维度是（，None）
    boxes -- NMS保留下的那些边框的坐标，维度是（4，None）
    classes -- NMS保留下的那些边框的种类的索引，维度是（，None）
    """
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    
    #tensorflow为我们提供来一个NMS函数，我们直接调用就可以来tf.image.non_max_suppression().
    #这个函数会返回NMS后保留下来的边框的索引
    nums_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold=iou_threshold)
    
    #通过上面的索引来分别获取被保留的边框的相关概率值，坐标以及种类的索引
    scores = K.gather(scores, nums_indices)
    boxes = K.gather(boxes, nums_indices)
    classes = K.gather(classes, nums_indices)
    
    return scores, boxes, classes

In [27]:
with tf.Session() as test_b:
    scores = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([54, 4], mean=1, stddev=4, seed = 1)
    classes = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))





scores[2] = 6.938395
boxes[2] = [-5.299932    3.1379814   4.450367    0.95942086]
classes[2] = -2.2452729
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


## 整合前两个函数，过滤掉最后多余的边框

In [30]:
def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold =.6, iou_threshold=.5):
    """
    参数：
    yolo_outputs -- yolo模型的输出结果
    image_shape -- 图像的原始像素，因为网上的yolo需要的图像像素是608，
                -- 而我们的原始图像是720 * 1280的，所以在输入到yolo
                -- 前会转成608的；同理，我们也会将yolo的输出结果转回到
                -- 720 * 1280
    max_boxes -- 希望最多识别出多少个边框
    score_threshold -- 概率值阈值
    iou_threshold -- 交并比阈值
    
    return:
    scores -- 最终保留下的那些边框的概率值，维度是（， None）
    boxes -- 最终保留下的那些边框的坐标，维度是（4， None）
    classes -- 最终保留下的那些边框的种类的索引，维度是（，None）
    """
    ### 将yolo输出结果分成3份，分别表示概率值，坐标，种类索引
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
    
    #因为yolo模型使用的坐标表示法是（x, y, w, h）,所以先将其转成（x1, y1, x2, y2）的形式
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    
    #使用我们前面实现的yolo_filter_boxes函数过滤掉概率值低于阈值的边框
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = score_threshold)
    
    # 将（608，608）转回（702，1280）
    boxes = scale_boxes(boxes, image_shape)
    
    #使用我们前面实现的yolo_non_max_suppression过滤掉重叠的边框
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold = iou_threshold)
    
    return scores, boxes, classes

In [31]:
with tf.Session() as test_b:
    yolo_outputs = (tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
    scores, boxes, classes = yolo_eval(yolo_outputs)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

scores[2] = 138.79124
boxes[2] = [1292.3297  -278.52167 3876.9893  -835.56494]
classes[2] = 54
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


# 使用yolo模型进行车辆探测
结合github上训练好的yolo模型以及我们自己实现的过滤函数来进行车辆探测

## 创建一个graph

In [32]:
sess = K.get_session()

## 定义种类以及anchor box和像素

In [33]:
class_names = read_classes('model_data/coco_classes.txt')
anchors = read_anchors('model_data/yolo_anchors.txt')
image_shape =(720. , 1280.)

## 加载已经训练好了的yolo模型
如果执行这句话时出现错误提示‘The kernel appears to have died. it will restart automatically’,那么你就要重新生成与自己开发环境匹配的yolo.h5

In [36]:
yolo_model = load_model("model_data/yolo.h5")
#上面已经报错，SystemError：unknown opcode，是因为直接跑了别人跑出来的模型
#需要重新生成与自己开发环境匹配的yolo.h5,步骤如下:
#1.将本文档目录下的YAD2Kmaster文件夹复制到c盘下
#2.打开Anaconda prompt
#3.执行activate tensorflow
#4.执行cd C:\YAD2Kmaster命令
#5.执行python yad2k.py yolov2.cfg yolov2.weights model_data/yolo.h5
#6.用c盘的YAD2Kmaster文件夹下的model_data替换掉本文档目录下的model_data

/Users/mac/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [37]:
yolo_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 608, 608, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 608, 608, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

## 将yolo模型的输出结果转换成我们需要的格式
因为yolo_model输出的维度是(m, 19, 19, 5, 85),所以我们要转换一下，这样好满足我们的yolo_eval函数的输入格式要求

In [38]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

## 过滤边框

yolo_model输出的yolo_outputs里面包含了它探测到的所有的边框。下面使用我们自己实现的过滤函数yolo_eval
来过滤掉多余的边框

In [39]:
socres, boxes, classes = yolo_eval(yolo_outputs, image_shape)

## 开始探测图片

In [44]:
# 使用前面构建好的graph来探测图片中的车辆
def predict(sess, image_file):
    """
    参数:
    sess -- 前面我们创建的包含了YOLO模型和过滤函数的graph
    image_file -- 待探测的图像
    """

    # 将图片读取出来，并且转换成608像素
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))

    # 运行我们之前构建好的graph
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], feed_dict={yolo_model.input: image_data, K.learning_phase(): 0})


    # 打印出找到了几个边框
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    # 为种类分配颜色
    colors = generate_colors(class_names)
    # 将找到的边框在图片上画出来
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    image.save(os.path.join("out", image_file), quality=90)
    # 将画出边框的图片显示出来
    output_image = scipy.misc.imread(os.path.join("out", image_file))
    imshow(output_image)
    
    return out_scores, out_boxes, out_classes

In [45]:
out_scores, out_boxes, out_classes = predict(sess, "test.jpg")

FailedPreconditionError: Attempting to use uninitialized value Variable_1
	 [[node Variable_1/read (defined at /Users/mac/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:402) ]]

Original stack trace for 'Variable_1/read':
  File "/Users/mac/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/mac/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/mac/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/mac/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/mac/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/Users/mac/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever(